In [ ]:
import os
current_working_directory = os.getcwd()
print(current_working_directory)

import numpy as np 

import torch 
import torch.nn as nn 
import torch.optim as optim

# import torch_optimizer as optim_custom
from torch.utils.data import Dataset, DataLoader
from math_utils.bernstein_torch import bernstein_coeff_ordern_new
import scipy.io as sio

# from models.mlp_qp_vis_aware_2 import MLP, vis_aware_track_net, PointNet
# import pol_matrix_comp
from tqdm import trange,tqdm

#from mlp_manipulator import MLP, MLPProjectionFilter

/home/alinjar/simple_training_warm_start


In [5]:
# Generating P matrix
t_fin = 20.0
num = 100
tot_time = torch.linspace(0, t_fin, num)
tot_time_copy = tot_time.reshape(num, 1)
P, Pdot, Pddot = bernstein_coeff_ordern_new(10, tot_time_copy[0], tot_time_copy[-1], tot_time_copy)


# P_diag = torch.block_diag(P, P)
# Pdot_diag = torch.block_diag(Pdot, Pdot)

# Pddot_diag = torch.block_diag(Pddot, Pddot)
nvar_single = P.size(dim = 1) 
num_dof = 1
nvar = nvar_single*num_dof
#print("P", P)


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")


Using cuda device


In [11]:

data = np.load("data/sample_dataset_final.npz")

xi_filtered = data['xi_filtered']
xi_samples = data['xi_samples']
state_terms = data['state_terms']

# Taking the last CEM iteration for samples
xi_samples = xi_samples[:, -1]
xi_filtered = xi_filtered[:, -1]


xi_filtered = xi_filtered.reshape(-1, xi_filtered.shape[2]) 
xi_samples = xi_samples.reshape(-1, xi_samples.shape[2])
state_terms = state_terms.reshape(-1, state_terms.shape[2])

datasize = state_terms.shape[0]

print("xi_filtered", xi_filtered.shape)
print("xi_samples", xi_samples.shape)
print("state_terms", state_terms.shape)

xi_filtered (974000, 66)
xi_samples (974000, 66)
state_terms (974000, 30)


In [12]:

print(os.getcwd())


/home/alinjar/simple_training_warm_start


In [13]:

init_state =state_terms

c_samples_input = xi_samples


inp = np.hstack((init_state, c_samples_input))


inp_mean, inp_std = inp.mean(), inp.std()

print("inp", inp.shape)

inp (974000, 96)


In [14]:
class TrajDataset(Dataset):
	"""Expert Trajectory Dataset."""
	def __init__(self, inp, init_state, c_samples_input):
		
		# input
		self.inp = inp
		# State Data
		self.init_state = init_state
		
		self.c_samples_input = c_samples_input
	
	def __len__(self):
		return len(self.inp)    
			
	def __getitem__(self, idx):
		
		# Inputs
		init_state = self.init_state[idx]
		
		c_samples_input = self.c_samples_input[idx]
  
		inp = self.inp[idx]
		
				 
		return torch.tensor(inp).float(), torch.tensor(init_state).float(), torch.tensor(c_samples_input).float()

In [15]:
batch_size = 10000 

# Using PyTorch Dataloader
train_dataset = TrajDataset(inp, init_state, c_samples_input)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, drop_last=True)

In [16]:
num_batch = train_loader.batch_size

P = P.to(device) 
Pdot = Pdot.to(device)
Pddot = Pddot.to(device)

# CVAE input
enc_inp_dim = np.shape(inp)[1] 
mlp_inp_dim = enc_inp_dim
hidden_dim = 1024
mlp_out_dim = 4*nvar#+3*num_constraint (lambda- 0:3*nvar, c_samples- 3*nvar:4*nvar)
#print(mlp_out_dim)



In [ ]:

mlp =  MLP(mlp_inp_dim, hidden_dim, mlp_out_dim)

#print(mlp)
print("P", P.size())

model = MLPProjectionFilter(P, Pdot, Pddot, mlp, num_batch, inp_mean, inp_std, t_fin).to(device)

print(type(model))